In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import math

In [3]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

pensCheckouts = pd.read_sql('SELECT * FROM PensCheckouts', con=db_connection)
pensCheckouts['start'] = pd.to_datetime(pensCheckouts['start'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
pensCheckouts['start'].replace(0, np.nan, inplace=True)
pensCheckouts['end'].replace(0, np.nan, inplace=True)
pensCheckouts['end'] = pd.to_datetime(pensCheckouts['end'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
pensCheckouts['start_ip'].replace(0, np.nan, inplace=True)
pensCheckouts['end_ip'].replace(0, np.nan, inplace=True)
pensCheckouts['start_ip'] = pensCheckouts['start_ip'].astype('category')
pensCheckouts['end_ip'] = pensCheckouts['end_ip'].astype('category')
pensCheckouts.set_index('id', inplace=True)
pensCheckouts.sort_index(inplace=True)

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [4]:
pensCheckouts

,pen_id,visit_id,start,end,start_ip,end_ip
id,,,,,,
68762351,8,793401,2014-10-23 11:36:51-04:00,2014-10-24 20:44:42-04:00,NaN,NaN
68762353,3,793449,2014-10-23 12:25:42-04:00,2014-10-24 20:44:42-04:00,NaN,NaN
68764535,9,793527,2014-10-23 22:04:59-04:00,2014-10-24 20:44:42-04:00,NaN,NaN
68764585,2,793571,2014-10-24 11:05:23-04:00,2014-10-27 13:53:29-04:00,NaN,NaN
68764655,7,793681,2014-10-24 20:02:19-04:00,2014-10-27 13:53:29-04:00,NaN,NaN
...,...,...,...,...,...,...
132777685,1520099,132777683,2020-07-02 16:22:28-04:00,2020-07-02 16:22:44-04:00,1.198075e+09,1.198075e+09
132794703,1520099,132794701,2020-07-06 13:42:18-04:00,2020-07-06 13:43:00-04:00,1.198075e+09,1.198075e+09
132806721,1520099,132806719,2020-07-09 09:12:28-04:00,2020-07-09 09:12:46-04:00,1.198075e+09,1.198075e+09


## what is the average time for pen uses

In [7]:
def checkout_average_time():
    pensCheckouts['time_spent'] = pensCheckouts['end'] - pensCheckouts['start']
    less_day = pensCheckouts['time_spent'] < '1day'
    less_day_df = pensCheckouts[less_day]
    min_spent = less_day_df["time_spent"].map(lambda timedelta: timedelta.total_seconds() / 60)
    min_spent = min_spent.dropna().map(lambda seconds: math.floor(seconds))
    min_spent = min_spent.to_frame()
    more_than_five_min = min_spent[min_spent['time_spent'] > 5]
    return more_than_five_min['time_spent'].mean()

In [8]:
checkout_average_time()

126.90362885266111

## how many Pens per year

In [9]:
def number_pens_per_year(year):
    yearly_pensCheckouts = pensCheckouts[pensCheckouts['start'].dt.year == year]
    return yearly_pensCheckouts['pen_id'].nunique()

In [10]:
number_pens_per_year(2015)

1694